#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [1]:
import pandas as pd
import pymongo
from bson.regex import Regex

In [2]:
client = pymongo.MongoClient('mongo')
database = client['exercices']
collection = database['kickstarter']

In [3]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head()

/root/.local/share/virtualenvs/code-AFz48Hjb/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (6,8,10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000,2016-07-01 21:55:54,7534,successful,254,US,3796,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000,2017-08-28 15:47:02,16298,successful,367,US,2670,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000,2016-01-25 17:37:10,1,failed,1,CA,0.708148,0.738225


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [4]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

## Question 0

### Netoyer les données

In [5]:
# Conversion des types des colonnes 'launched', Supression des valeurs manquantes 
df_ks['launched'] = pd.to_datetime(df_ks['launched'], errors='coerce')
df_ks = df_ks.dropna(subset = ['launched'])

# On renomme la colonne ID
df_ks = df_ks.rename(columns = {"ID": "_id"})

### Importer les données

In [6]:
collection.delete_many({})
collection.insert_many(df_ks.to_dict(orient='records'))

## Question 1  

In [7]:
# Trie selon les promesses de dons decroissantes
dons = collection.find({}, {"name":1,'_id':0}).sort("pledged", -1).limit(5)
print("Les promesses de dons ont été les plus fortes pour : ",list(dons))

Les promesses de dons ont été les plus fortes pour :  [{'name': 'True North ï¿½ Repurposed Sailcloth Products'}, {'name': 'Scott Davis - Solo Record'}, {'name': 'Polco: A Digital Platform for Better Civic Communication'}, {'name': '2016 Gaming Calendar'}, {'name': 'The Girl in The Mountain - Film'}]


## Question 2

In [8]:
# Comptage du nombre d'éléments dont le but est atteint
goal = collection.count_documents({'state' : 'successful'})
print(str(goal) + "projets ont atteint leur but")

53040projets ont atteint leur but


## Question 3

In [9]:
# Groupement par catégorie de projet 
projet_par_caté= collection.aggregate([{"$group" : {"_id" : "$category","Nombre de projet" : {"$sum" : 1}}}])
list(projet_par_caté)

[{'_id': 'Jewelry', 'Nombre de projet': 472},
 {'_id': 'Drinks', 'Nombre de projet': 995},
 {'_id': 'Food Trucks', 'Nombre de projet': 687},
 {'_id': 'Bacon', 'Nombre de projet': 78},
 {'_id': 'Printing', 'Nombre de projet': 83},
 {'_id': 'Nature', 'Nombre de projet': 219},
 {'_id': 'Dance', 'Nombre de projet': 901},
 {'_id': 'Musical', 'Nombre de projet': 367},
 {'_id': 'Narrative Film', 'Nombre de projet': 2099},
 {'_id': 'Immersive', 'Nombre de projet': 131},
 {'_id': 'Country & Folk', 'Nombre de projet': 1790},
 {'_id': 'Games', 'Nombre de projet': 1331},
 {'_id': 'Taxidermy', 'Nombre de projet': 7},
 {'_id': 'Food', 'Nombre de projet': 4612},
 {'_id': 'Blues', 'Nombre de projet': 113},
 {'_id': 'Translations', 'Nombre de projet': 76},
 {'_id': 'Comic Books', 'Nombre de projet': 1032},
 {'_id': 'Television', 'Nombre de projet': 401},
 {'_id': 'Literary Spaces', 'Nombre de projet': 10},
 {'_id': 'Workshops', 'Nombre de projet': 59},
 {'_id': 'Public Art', 'Nombre de projet': 1248},


## Question 4

In [10]:
import datetime
year = datetime.datetime.strptime("2016-01-01",'%Y-%m-%d')
collection.count_documents({"$and" :[{"launched" : {"$lte" : year}},{"country" : "FR"}]})

330

In [11]:
collection.find_one({})

{'_id': 872782264,
 'name': 'Scott Cooper\'s Solo CD "A Leg Trick" (Canceled)',
 'category': 'Rock',
 'main_category': 'Music',
 'currency': 'USD',
 'deadline': '2011-09-16',
 'goal': 2000.0,
 'launched': datetime.datetime(2011, 8, 17, 6, 31, 31),
 'pledged': 1145.0,
 'state': 'canceled',
 'backers': 24,
 'country': 'US',
 'usd pledged': 1145.0,
 'usd_pledged_real': 1145.0}

## Question 5

In [12]:
# On compte le nombre de projets venant des états Unis ("US") avec la contrainte "usd_pledged_real">200 000
expensive = collection.count_documents({"$and":[{"country":"US"},{"usd_pledged_real":{"$gte":200000}}]})
print(str(expensive)+" projets ont demandé plus de 200 000 dollars aux USA")

658 projets ont demandé plus de 200 000 dollars aux USA


## Question 6 

In [13]:
# On compte le nombre de projet avec le mot "Sport"
with_sport = collection.count_documents({"name" : Regex("Sport")})
print(str(with_sport)+" projets ont sport dans leur nom")

323 projets ont sport dans leur nom
